### CTD

In [1]:
# Download Link: https://ctdbase.org/reports/CTD_chemicals_diseases.csv.gz
# Download Date: 2025-03-21
# Download Version: 2025-03

import pandas as pd
from io import StringIO
# Read the lines from the file
with open('CTD_chemicals_diseases.csv', 'r') as f:
    lines = f.readlines()

# Extract column names
columns = []
for line in lines:
    if line.startswith('# Fields'):
        # Get the following line which contains column names
        columns = lines[lines.index(line) + 1].strip()[2:].split(',')
        break

# Filter out comment lines
data_lines = [line for line in lines if not line.startswith('#')]

# Create a DataFrame from the filtered lines
data_str = '\n'.join(data_lines)
ctd = pd.read_csv(StringIO(data_str), names=columns)

ctd

,ChemicalName,ChemicalID,CasRN,DiseaseName,DiseaseID,DirectEvidence,InferenceGeneSymbol,InferenceScore,OmimIDs,PubMedIDs
0,06-Paris-LA-66 protocol,C046983,NaN,Precursor Cell Lymphoblastic Leukemia-Lymphoma,MESH:D054198,therapeutic,NaN,NaN,NaN,4519131
1,10074-G5,C534883,NaN,Adenocarcinoma,MESH:D000230,NaN,MYC,4.07,NaN,26432044
2,10074-G5,C534883,NaN,Adenocarcinoma of Lung,MESH:D000077192,NaN,MYC,4.30,NaN,26656844|27602772
3,10074-G5,C534883,NaN,Alopecia,MESH:D000505,NaN,AR,4.50,NaN,15902657
4,10074-G5,C534883,NaN,Androgen-Insensitivity Syndrome,MESH:D013734,NaN,AR,6.87,300068|312300,1303262|8281139
...,...,...,...,...,...,...,...,...,...,...
9329078,zymosterol,C015582,128-33-6,"Bone Diseases, Metabolic",MESH:D001851,NaN,CYP27A1,5.92,NaN,24381012
9329079,zymosterol,C015582,128-33-6,Cholestasis,MESH:D002779,NaN,CYP27A1,4.82,NaN,15795599
9329080,zymosterol,C015582,128-33-6,Liver Cirrhosis,MESH:D008103,NaN,CYP27A1,4.87,NaN,24381012
9329081,zymosterol,C015582,128-33-6,"MICROCEPHALY, CONGENITAL CATARACT, AND PSORIAS...",OMIM:616834,NaN,MSMO1,7.80,616834,NaN


In [2]:
ctd_filter = ctd[['ChemicalID', 'DiseaseID', 'OmimIDs']]
ctd_filter[['MESH', 'OMIM']] = ctd_filter['DiseaseID'].str.extract(r'(MESH:[^\s]+)|(OMIM:[^\s]+)')
ctd_filter

C:\Users\shunn\AppData\Local\Temp\ipykernel_35772\2994966874.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ctd_filter[['MESH', 'OMIM']] = ctd_filter['DiseaseID'].str.extract(r'(MESH:[^\s]+)|(OMIM:[^\s]+)')
C:\Users\shunn\AppData\Local\Temp\ipykernel_35772\2994966874.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ctd_filter[['MESH', 'OMIM']] = ctd_filter['DiseaseID'].str.extract(r'(MESH:[^\s]+)|(OMIM:[^\s]+)')


,ChemicalID,DiseaseID,OmimIDs,MESH,OMIM
0,C046983,MESH:D054198,NaN,MESH:D054198,NaN
1,C534883,MESH:D000230,NaN,MESH:D000230,NaN
2,C534883,MESH:D000077192,NaN,MESH:D000077192,NaN
3,C534883,MESH:D000505,NaN,MESH:D000505,NaN
4,C534883,MESH:D013734,300068|312300,MESH:D013734,NaN
...,...,...,...,...,...
9329078,C015582,MESH:D001851,NaN,MESH:D001851,NaN
9329079,C015582,MESH:D002779,NaN,MESH:D002779,NaN
9329080,C015582,MESH:D008103,NaN,MESH:D008103,NaN
9329081,C015582,OMIM:616834,616834,NaN,OMIM:616834


In [3]:
def merge_string_columns(df, columns, merge_name, separator=';'):
    def merge_strings(row):
        combined = set()
        for column in columns:
            if pd.notnull(row[column]):
                combined.update(row[column].split(separator))
        return separator.join(combined)
    
    # Apply the function to each row and create a new column
    combined_column_name = merge_name
    df[combined_column_name] = df.apply(merge_strings, axis=1)
    df.drop(columns=columns, inplace=True)

    return df

ctd_filter['OmimIDs'] = ctd_filter['OmimIDs'].str.replace('|', ';').replace('OMIM:', '')
ctd_filter = merge_string_columns(ctd_filter, ['OmimIDs', 'OMIM'], 'omim')
ctd_filter

C:\Users\shunn\AppData\Local\Temp\ipykernel_35772\3872167810.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ctd_filter['OmimIDs'] = ctd_filter['OmimIDs'].str.replace('|', ';').replace('OMIM:', '')
C:\Users\shunn\AppData\Local\Temp\ipykernel_35772\3872167810.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[combined_column_name] = df.apply(merge_strings, axis=1)
C:\Users\shunn\AppData\Local\Temp\ipykernel_35772\3872167810.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a 

,ChemicalID,DiseaseID,MESH,omim
0,C046983,MESH:D054198,MESH:D054198,
1,C534883,MESH:D000230,MESH:D000230,
2,C534883,MESH:D000077192,MESH:D000077192,
3,C534883,MESH:D000505,MESH:D000505,
4,C534883,MESH:D013734,MESH:D013734,300068;312300
...,...,...,...,...
9329078,C015582,MESH:D001851,MESH:D001851,
9329079,C015582,MESH:D002779,MESH:D002779,
9329080,C015582,MESH:D008103,MESH:D008103,
9329081,C015582,OMIM:616834,NaN,OMIM:616834;616834


### BioMedGraphica ID

In [4]:
import pandas as pd
import os
from pathlib import Path

current_working_dir = Path(os.getcwd()).resolve()
grandparent_dir = current_working_dir.parent.parent.parent
target_dir_exposure = grandparent_dir / 'BioMedGraphica' / 'Entity' / 'Exposure' / 'BioMedGraphica_Exposure.csv'
target_dir_disease = grandparent_dir / 'BioMedGraphica' / 'Entity' / 'Disease' / 'BioMedGraphica_Disease.csv'
biomedgraphica_exposure = pd.read_csv(target_dir_exposure, dtype=str)
biomedgraphica_disease = pd.read_csv(target_dir_disease, dtype=str)

### CTD Mapping

In [5]:
mesh_individual = biomedgraphica_exposure[['MeSH_ID', 'BioMedGraphica_ID']]
mesh_individual.dropna(subset=['MeSH_ID'], inplace=True)
mesh_individual = mesh_individual.assign(MeSH_ID=mesh_individual['MeSH_ID'].str.split(';')).explode('MeSH_ID')

mesh_to_individual = mesh_individual.groupby('MeSH_ID')['BioMedGraphica_ID'].apply(lambda x: ';'.join(x.dropna().unique())).to_dict()

ctd_filter['From_ID'] = ctd_filter['ChemicalID'].map(mesh_to_individual)
ctd_filter

C:\Users\shunn\AppData\Local\Temp\ipykernel_35772\1095249926.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mesh_individual.dropna(subset=['MeSH_ID'], inplace=True)
C:\Users\shunn\AppData\Local\Temp\ipykernel_35772\1095249926.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ctd_filter['From_ID'] = ctd_filter['ChemicalID'].map(mesh_to_individual)


,ChemicalID,DiseaseID,MESH,omim,From_ID
0,C046983,MESH:D054198,MESH:D054198,,NaN
1,C534883,MESH:D000230,MESH:D000230,,NaN
2,C534883,MESH:D000077192,MESH:D000077192,,NaN
3,C534883,MESH:D000505,MESH:D000505,,NaN
4,C534883,MESH:D013734,MESH:D013734,300068;312300,NaN
...,...,...,...,...,...
9329078,C015582,MESH:D001851,MESH:D001851,,NaN
9329079,C015582,MESH:D002779,MESH:D002779,,NaN
9329080,C015582,MESH:D008103,MESH:D008103,,NaN
9329081,C015582,OMIM:616834,NaN,OMIM:616834;616834,NaN


In [6]:
omim_individual = biomedgraphica_disease[['OMIM_ID', 'BioMedGraphica_ID']]
omim_individual.dropna(subset=['OMIM_ID'], inplace=True)
omim_individual = omim_individual.assign(OMIM_ID=omim_individual['OMIM_ID'].str.split(';')).explode('OMIM_ID')

omim_to_individual = omim_individual.groupby('OMIM_ID')['BioMedGraphica_ID'].apply(lambda x: ';'.join(x.dropna().unique())).to_dict()

ctd_filter['omim'] = ctd_filter['omim'].str.split(';')
ctd_filter = ctd_filter.explode('omim')
ctd_filter['OMIM'] = ctd_filter['omim'].map(omim_to_individual)
ctd_filter

C:\Users\shunn\AppData\Local\Temp\ipykernel_35772\329021157.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  omim_individual.dropna(subset=['OMIM_ID'], inplace=True)
C:\Users\shunn\AppData\Local\Temp\ipykernel_35772\329021157.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ctd_filter['omim'] = ctd_filter['omim'].str.split(';')


,ChemicalID,DiseaseID,MESH,omim,From_ID,OMIM
0,C046983,MESH:D054198,MESH:D054198,,NaN,NaN
1,C534883,MESH:D000230,MESH:D000230,,NaN,NaN
2,C534883,MESH:D000077192,MESH:D000077192,,NaN,NaN
3,C534883,MESH:D000505,MESH:D000505,,NaN,NaN
4,C534883,MESH:D013734,MESH:D013734,300068,NaN,BMG_DS003142
...,...,...,...,...,...,...
9329079,C015582,MESH:D002779,MESH:D002779,,NaN,NaN
9329080,C015582,MESH:D008103,MESH:D008103,,NaN,NaN
9329081,C015582,OMIM:616834,NaN,OMIM:616834,NaN,NaN
9329081,C015582,OMIM:616834,NaN,616834,NaN,BMG_DS071730


In [7]:
mesh_individual = biomedgraphica_disease[['MeSH_ID', 'BioMedGraphica_ID']]
mesh_individual.dropna(subset=['MeSH_ID'], inplace=True)
mesh_individual = mesh_individual.assign(MeSH_ID=mesh_individual['MeSH_ID'].str.split(';')).explode('MeSH_ID')

mesh_to_individual = mesh_individual.groupby('MeSH_ID')['BioMedGraphica_ID'].apply(lambda x: ';'.join(x.dropna().unique())).to_dict()

ctd_filter['MESH'].replace('MESH:', '', regex=True, inplace=True)
ctd_filter['mesh'] = ctd_filter['MESH'].map(mesh_to_individual)
ctd_filter

C:\Users\shunn\AppData\Local\Temp\ipykernel_35772\3758706789.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mesh_individual.dropna(subset=['MeSH_ID'], inplace=True)
C:\Users\shunn\AppData\Local\Temp\ipykernel_35772\3758706789.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  ctd_filter['MESH'].replace('MESH:', '', regex=True, inplace=Tr

,ChemicalID,DiseaseID,MESH,omim,From_ID,OMIM,mesh
0,C046983,MESH:D054198,D054198,,NaN,NaN,BMG_DS079565
1,C534883,MESH:D000230,D000230,,NaN,NaN,BMG_DS077905
2,C534883,MESH:D000077192,D000077192,,NaN,NaN,BMG_DS077782
3,C534883,MESH:D000505,D000505,,NaN,NaN,BMG_DS000114;BMG_DS003691;BMG_DS005696;BMG_DS0...
4,C534883,MESH:D013734,D013734,300068,NaN,BMG_DS003142,BMG_DS003142;BMG_DS010022;BMG_DS029569;BMG_DS0...
...,...,...,...,...,...,...,...
9329079,C015582,MESH:D002779,D002779,,NaN,NaN,BMG_DS000596
9329080,C015582,MESH:D008103,D008103,,NaN,NaN,BMG_DS001844;BMG_DS006709
9329081,C015582,OMIM:616834,NaN,OMIM:616834,NaN,NaN,NaN
9329081,C015582,OMIM:616834,NaN,616834,NaN,BMG_DS071730,NaN


In [8]:
def merge_string_columns(df, columns, merge_name, separator=';'):
    def merge_strings(row):
        combined = set()
        for column in columns:
            if pd.notnull(row[column]):
                combined.update(row[column].split(separator))
        return separator.join(combined)
    
    # Apply the function to each row and create a new column
    combined_column_name = merge_name
    df[combined_column_name] = df.apply(merge_strings, axis=1)
    df.drop(columns=columns, inplace=True)
    
    return df

ctd_filter.replace('', pd.NA, inplace=True)
ctd_filter = merge_string_columns(ctd_filter, ['OMIM', 'mesh'], 'To_ID')
ctd_filter

,ChemicalID,DiseaseID,MESH,omim,From_ID,To_ID
0,C046983,MESH:D054198,D054198,<NA>,NaN,BMG_DS079565
1,C534883,MESH:D000230,D000230,<NA>,NaN,BMG_DS077905
2,C534883,MESH:D000077192,D000077192,<NA>,NaN,BMG_DS077782
3,C534883,MESH:D000505,D000505,<NA>,NaN,BMG_DS003691;BMG_DS000114;BMG_DS060025;BMG_DS0...
4,C534883,MESH:D013734,D013734,300068,NaN,BMG_DS029569;BMG_DS010022;BMG_DS003142;BMG_DS0...
...,...,...,...,...,...,...
9329079,C015582,MESH:D002779,D002779,<NA>,NaN,BMG_DS000596
9329080,C015582,MESH:D008103,D008103,<NA>,NaN,BMG_DS006709;BMG_DS001844
9329081,C015582,OMIM:616834,NaN,OMIM:616834,NaN,
9329081,C015582,OMIM:616834,NaN,616834,NaN,BMG_DS071730


In [9]:
exposure_disease = ctd_filter[['From_ID', 'To_ID']]
exposure_disease.replace('', pd.NA, inplace=True)
exposure_disease.dropna(subset=['From_ID'], inplace=True)
exposure_disease.dropna(subset=['To_ID'], inplace=True)
exposure_disease.drop_duplicates(inplace=True)
exposure_disease

C:\Users\shunn\AppData\Local\Temp\ipykernel_35772\832042014.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exposure_disease.replace('', pd.NA, inplace=True)
C:\Users\shunn\AppData\Local\Temp\ipykernel_35772\832042014.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exposure_disease.dropna(subset=['From_ID'], inplace=True)
C:\Users\shunn\AppData\Local\Temp\ipykernel_35772\832042014.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-co

,From_ID,To_ID
6332,BMG_EP0060,BMG_DS077693
6333,BMG_EP0060,BMG_DS001687;BMG_DS036641
6334,BMG_EP0060,BMG_DS077905
6335,BMG_EP0060,BMG_DS077782
6336,BMG_EP0060,BMG_DS035558;BMG_DS005695
...,...,...
9311545,BMG_EP1159,BMG_DS066199;BMG_DS002733
9311546,BMG_EP1159,BMG_DS078802
9311547,BMG_EP1159,BMG_DS002873
9311548,BMG_EP1159,BMG_DS077787


In [10]:
exposure_disease['From_ID'] = exposure_disease['From_ID'].str.split(';')
exposure_disease = exposure_disease.explode('From_ID')
exposure_disease['To_ID'] = exposure_disease['To_ID'].str.split(';')
exposure_disease = exposure_disease.explode('To_ID')
exposure_disease.drop_duplicates(inplace=True)
exposure_disease

C:\Users\shunn\AppData\Local\Temp\ipykernel_35772\4103578911.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exposure_disease['From_ID'] = exposure_disease['From_ID'].str.split(';')


,From_ID,To_ID
6332,BMG_EP0060,BMG_DS077693
6333,BMG_EP0060,BMG_DS001687
6333,BMG_EP0060,BMG_DS036641
6334,BMG_EP0060,BMG_DS077905
6335,BMG_EP0060,BMG_DS077782
...,...,...
9311545,BMG_EP1159,BMG_DS002733
9311546,BMG_EP1159,BMG_DS078802
9311547,BMG_EP1159,BMG_DS002873
9311548,BMG_EP1159,BMG_DS077787


### Exposure-Disease Relation

In [11]:
exposure_disease['Type'] = 'Exposure-Disease'
exposure_disease['Source'] = 'CTD'

max_length = len(str(len(exposure_disease)))
exposure_disease['BioMedGraphica_ID'] = ['BMG_ED_EPDS' + str(i).zfill(max_length) for i in range(1, len(exposure_disease) + 1)]
columns = ['BioMedGraphica_ID'] + [col for col in exposure_disease.columns if col != 'BioMedGraphica_ID']  # re-order columns
exposure_disease = exposure_disease[columns]
exposure_disease

,BioMedGraphica_ID,From_ID,To_ID,Type,Source
6332,BMG_ED_EPDS000001,BMG_EP0060,BMG_DS077693,Exposure-Disease,CTD
6333,BMG_ED_EPDS000002,BMG_EP0060,BMG_DS001687,Exposure-Disease,CTD
6333,BMG_ED_EPDS000003,BMG_EP0060,BMG_DS036641,Exposure-Disease,CTD
6334,BMG_ED_EPDS000004,BMG_EP0060,BMG_DS077905,Exposure-Disease,CTD
6335,BMG_ED_EPDS000005,BMG_EP0060,BMG_DS077782,Exposure-Disease,CTD
...,...,...,...,...,...
9311545,BMG_ED_EPDS979776,BMG_EP1159,BMG_DS002733,Exposure-Disease,CTD
9311546,BMG_ED_EPDS979777,BMG_EP1159,BMG_DS078802,Exposure-Disease,CTD
9311547,BMG_ED_EPDS979778,BMG_EP1159,BMG_DS002873,Exposure-Disease,CTD
9311548,BMG_ED_EPDS979779,BMG_EP1159,BMG_DS077787,Exposure-Disease,CTD


In [12]:
import os
from pathlib import Path

# get the current working directory
current_working_dir = Path(os.getcwd()).resolve()

# get the output directory
grandparent_dir = current_working_dir.parent.parent.parent

target_folder = grandparent_dir / 'BioMedGraphica' / 'Relation' / 'Exposure-Disease'
if not target_folder.exists():
    target_folder.mkdir(parents=True)
    print(f"Folder {target_folder} has been created.")

output_file_path = target_folder / 'BioMedGraphica_Exposure_Disease.csv'
exposure_disease.to_csv(output_file_path, index=False)
print(f"Data has been saved to {output_file_path}")

Data has been saved to D:\RA\BMG\BioMedGraphica\Relation\Exposure-Disease\BioMedGraphica_Exposure_Disease.csv
